In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt 
import numpy as np 
import os

In [0]:
DATA_DIR = "/gdrive/My Drive/mask_detection"
BATCH_SIZE = 32
LEARNING_RATE = 1e-4

In [0]:
data = []
labels = []

for root, dirs, files in os.walk(DATA_DIR):
	for file in files:
		imgPath = os.path.join(root, file)
		label = os.path.basename(os.path.dirname(imgPath))

		img = load_img(imgPath, target_size = (224, 224))
		img = img_to_array(img)
		img = preprocess_input(img)

		data.append(img)
		labels.append(label)

data = np.array(data, dtype = "float32")
labels = np.array(labels)

lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

(x_train, x_test, y_train, y_test) = train_test_split(
                            data, labels, 
													  test_size = 0.2, 
													  stratify = labels, 
													  random_state = 42)

In [0]:
trainAug = ImageDataGenerator(
	rotation_range = 30,
	zoom_range = 0.2,
	width_shift_range = 0.2,
	height_shift_range = 0.2,
	horizontal_flip = True,
	fill_mode = "nearest")

In [23]:
mobileNet = MobileNetV2(
	weights = "imagenet", 
	include_top = False, 
	input_tensor = Input(shape = (224, 224, 3)))

mobileNet.summary()

for layer in mobileNet.layers:
	layer.trainable = False

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [27]:
model = Sequential()

model.add(mobileNet)

model.add(Conv2D(256, (3, 3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, 2))

model.add(Flatten())

model.add(Dense(64, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(2, activation = 'softmax'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 5, 256)         2949376   
_________________________________________________________________
batch_normalization_1 (Batch (None, 5, 5, 256)         1024      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 2, 2, 256)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               131200    
_________________________________________________________________
batch_normalization_2 (Batch (None, 128)              

In [0]:
optimizer = Adam(lr = 0.001)

model.compile(
	loss = "binary_crossentropy",
	optimizer = optimizer,
	metrics = ["accuracy"])

In [31]:
earlyStop = EarlyStopping(
    monitor = 'val_loss',
    patience = 3,
    verbose = 1,
    restore_best_weights = True)

H = model.fit(
	trainAug.flow(x_train, y_train, batch_size = BATCH_SIZE),
	steps_per_epoch = len(x_train) // BATCH_SIZE,
	validation_data = (x_test, y_test),
	validation_steps = len(x_test) // BATCH_SIZE,
  callbacks = [earlyStop],
	epochs = 20)

Epoch 1/20
31/31 [==============================] - 11s 369ms/step - loss: 0.1072 - accuracy: 0.9636 - val_loss: 0.0566 - val_accuracy: 0.9922
Epoch 2/20
31/31 [==============================] - 11s 339ms/step - loss: 0.0238 - accuracy: 0.9899 - val_loss: 0.0075 - val_accuracy: 0.9961
Epoch 3/20
31/31 [==============================] - 10s 336ms/step - loss: 0.0279 - accuracy: 0.9919 - val_loss: 0.0767 - val_accuracy: 0.9805
Epoch 4/20
31/31 [==============================] - 10s 336ms/step - loss: 0.0305 - accuracy: 0.9889 - val_loss: 0.0100 - val_accuracy: 0.9961
Epoch 5/20
31/31 [==============================] - 11s 340ms/step - loss: 0.0315 - accuracy: 0.9879 - val_loss: 0.0038 - val_accuracy: 0.9961
Epoch 6/20
31/31 [==============================] - 10s 336ms/step - loss: 0.0437 - accuracy: 0.9879 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 7/20
31/31 [==============================] - 10s 336ms/step - loss: 0.0223 - accuracy: 0.9939 - val_loss: 6.9078e-04 - val_accuracy: 1.

In [32]:
predictions = model.predict(x_test, BATCH_SIZE)
predictions = np.argmax(predictions, axis = 1)
print(classification_report(y_test.argmax(axis = 1), predictions, target_names = lb.classes_))

              precision    recall  f1-score   support

        mask       1.00      1.00      1.00       138
     no mask       1.00      1.00      1.00       118

    accuracy                           1.00       256
   macro avg       1.00      1.00      1.00       256
weighted avg       1.00      1.00      1.00       256



In [0]:
model.save("mask_detector_model", save_format = 'h5')